# Amazon SageMaker Clarify Model Bias Monitor for Batch Transform - JSON Format

---

This notebook's CI test result for us-west-2 is as follows. CI test results in other regions can be found at the end of the notebook.

![This us-west-2 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://prod.us-west-2.tcx-beacon.docs.aws.dev/sagemaker-nb/us-west-2/deploy_and_monitor|sm-clarify_model_bias_monitor_batch_transform_json|sm-clarify_model_bias_monitor_batch_transform_json.ipynb)

---

## Runtime

This notebook takes approximately 60 minutes to run.

## Contents

* [Introduction](#Introduction)
* [General Setup](#General-Setup)
    * [Imports](#Imports)
    * [Handful of configuration](#Handful-of-configuration)
    * [Data files](#Data-files)
    * [SageMaker model](#SageMaker-model)
* [Batch Transform Job](#Batch-Transform-Job)
    * [Captured data](#Captured-data)
    * [Transform output](#Transform-output)
* [Ground Truth Data](#Ground-Truth-Data)
* [Model Bias Monitor](#Model-Bias-Monitor)
    * [Baselining job](#Baselining-job)
        * [Configurations](#Configurations)
        * [Kick off baselining job](#Kick-off-baselining-job)
    * [Monitoring Schedule](#Monitoring-Schedule)
        * [Wait for the first execution](#Wait-for-the-first-execution)
        * [Wait for the execution to finish](#Wait-for-the-execution-to-finish)
        * [Merged data](#Merged-data)
        * [Inspect execution results](#Inspect-execution-results)
* [Cleanup](#Cleanup)

## Introduction

[Amazon SageMaker Model Monitor](https://docs.aws.amazon.com/sagemaker/latest/dg/model-monitor.html) continuously monitors the quality of Amazon SageMaker machine learning models in production. It enables developers to set alerts for when there are deviations in the model quality. Early and pro-active detection of these deviations enables corrective actions, such as retraining models, auditing upstream systems, or fixing data quality issues without having to monitor models manually or build additional tooling. 

[Amazon SageMaker Clarify Model Bias Monitor](https://docs.aws.amazon.com/sagemaker/latest/dg/clarify-model-monitor-bias-drift.html) is a model monitor that helps data scientists and ML engineers monitor predictions for bias on a regular basis. Bias can be introduced or exacerbated in deployed ML models when the training data differs from the data that the model sees during deployment (that is, the live data). These kinds of changes in the live data distribution might be temporary (for example, due to some short-lived, real-world events) or permanent. In either case, it might be important to detect these changes. For example, the outputs of a model for predicting home prices can become biased if the mortgage rates used to train the model differ from current, real-world mortgage rates. With bias drift detection capabilities in model monitor, when SageMaker detects bias beyond a certain threshold, it automatically generates metrics that you can view in SageMaker Studio and through Amazon CloudWatch alerts. 

This notebook demonstrates the process for setting up a model monitor for continuous monitoring of bias drift of the data and model used by a regularly running [SageMaker Batch Transform](https://docs.aws.amazon.com/sagemaker/latest/dg/batch-transform.html) job. The model input and output are in [SageMaker JSON Lines dense format](https://docs.aws.amazon.com/sagemaker/latest/dg/cdf-inference.html#common-in-formats).

In general, you can use the model bias monitor for batch transform in this way,

1. Schedule a model bias monitor to monitor a data capture S3 location and a ground truth S3 location
1. Regularly run transform jobs with data capture enabled, the jobs save captured data to the data capture S3 URI
1. Regularly label the captured data, and then upload the ground truth labels to the ground truth S3 URI

The monitor executes processing jobs regularly to merge the captured data and ground truth data, do bias analysis for the merged data, and then generate analysis reports and publish metrics to CloudWatch.

## General Setup

The notebook uses the [SageMaker Python SDK](https://github.com/aws/sagemaker-python-sdk). The following cell upgrades the SDK and its dependencies. Then you may need to restart the kernel and rerun the notebook to pick up the up-to-date APIs, if the notebook is executed in the SageMaker Studio.

In [1]:
!pip install -U sagemaker
!pip install -U boto3
!pip install -U botocore


[notice] A new release of pip is available: 23.0.1 -> 23.3.2
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.0.1 -> 23.3.2
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.0.1 -> 23.3.2
[notice] To update, run: pip install --upgrade pip


### Imports

The following cell imports the APIs to be used by the notebook.

In [2]:
import sagemaker
import pandas as pd
import datetime
import json
import os
import pprint
import random
import time

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/zicanl/.config/sagemaker/config.yaml


### Handful of configuration

To begin, ensure that these prerequisites have been completed.

* Specify an AWS Region to host the model.
* Specify an IAM role to execute jobs.
* Define the S3 URIs that stores the model file, input data and output data. For demonstration purposes, this notebook uses the same bucket for them. In reality, they could be separated with different security policies.

In [3]:
sagemaker_session = sagemaker.Session()

region = sagemaker_session.boto_region_name
print(f"AWS region: {region}")

role = sagemaker.get_execution_role()
print(f"RoleArn: {role}")

# A different bucket can be used, but make sure the role for this notebook has
# the s3:PutObject permissions. This is the bucket into which the data is captured
bucket = sagemaker_session.default_bucket()
print(f"Demo Bucket: {bucket}")
prefix = sagemaker.utils.unique_name_from_base("sagemaker/DEMO-ClarifyModelMonitor")
print(f"Demo Prefix: {prefix}")
s3_key = f"s3://{bucket}/{prefix}"
print(f"Demo S3 key: {s3_key}")

data_capture_s3_uri = f"{s3_key}/data-capture"
ground_truth_s3_uri = f"{s3_key}/ground-truth"
transform_output_s3_uri = f"{s3_key}/transform-output"
baselining_output_s3_uri = f"{s3_key}/baselining-output"
monitor_output_s3_uri = f"{s3_key}/monitor-output"

print(f"The transform job will save the results to: {transform_output_s3_uri}")
print(f"The transform job will save the captured data to: {data_capture_s3_uri}")
print(f"You should upload the ground truth data to: {ground_truth_s3_uri}")
print(f"The baselining job will save the analysis results to: {baselining_output_s3_uri}")
print(f"The monitor will save the analysis results to: {monitor_output_s3_uri}")

AWS region: us-west-2
RoleArn: arn:aws:iam::678264136642:role/Admin
Demo Bucket: sagemaker-us-west-2-678264136642
Demo Prefix: sagemaker/DEMO-ClarifyModelMonitor-1705692244-1224
Demo S3 key: s3://sagemaker-us-west-2-678264136642/sagemaker/DEMO-ClarifyModelMonitor-1705692244-1224
The transform job will save the results to: s3://sagemaker-us-west-2-678264136642/sagemaker/DEMO-ClarifyModelMonitor-1705692244-1224/transform-output
The transform job will save the captured data to: s3://sagemaker-us-west-2-678264136642/sagemaker/DEMO-ClarifyModelMonitor-1705692244-1224/data-capture
You should upload the ground truth data to: s3://sagemaker-us-west-2-678264136642/sagemaker/DEMO-ClarifyModelMonitor-1705692244-1224/ground-truth
The baselining job will save the analysis results to: s3://sagemaker-us-west-2-678264136642/sagemaker/DEMO-ClarifyModelMonitor-1705692244-1224/baselining-output
The monitor will save the analysis results to: s3://sagemaker-us-west-2-678264136642/sagemaker/DEMO-ClarifyMode

### Data files

This example includes two dataset files, both in the JSON format.

In [4]:
train_dataset_path = "test_data/validation-dataset.json"
test_dataset_path = "test_data/test-dataset.json"
dataset_type = "application/json"

The train dataset has the features and the ground truth label (pointed to by the key "label"),

In [5]:
!head -n 5 $train_dataset_path

{"instances":[{"features":[41,2,220531,14,15,2,9,0,4,1,0,0,60,38],"label":1},{"features":[33,2,35378,9,13,2,11,5,4,0,0,0,45,38],"label":1},{"features":[36,2,223433,12,14,2,11,0,4,1,7688,0,50,38],"label":1},{"features":[40,2,220589,7,12,4,0,1,4,0,0,0,40,38],"label":0},{"features":[30,2,231413,15,10,2,2,0,4,1,0,0,40,38],"label":1},{"features":[33,4,218164,11,9,2,2,0,4,1,0,0,40,38],"label":0},{"features":[42,2,213464,15,10,2,2,0,4,1,0,0,40,38],"label":0},{"features":[20,2,247794,11,9,4,11,1,4,0,0,0,84,38],"label":0},{"features":[43,2,174575,15,10,0,0,1,4,1,0,0,45,38],"label":0},{"features":[42,4,54202,14,15,2,9,0,4,1,0,0,50,38],"label":1},{"features":[27,2,126060,11,9,4,3,1,4,0,0,0,40,38],"label":0},{"features":[25,2,182866,11,9,4,5,3,4,1,0,0,40,38],"label":0},{"features":[43,2,302041,11,9,4,0,1,2,0,0,0,40,38],"label":0},{"features":[30,2,91145,11,9,4,5,4,4,1,0,0,55,38],"label":0},{"features":[41,2,648223,3,2,3,4,4,4,1,0,0,40,25],"label":0},{"features":[60,2,101096,10,16,4,9,1,4,0,0,0,65,

The test dataset only has features.

In [6]:
!head -n 5 $test_dataset_path

{"instances":[{"features":[28,2,133937,9,13,2,0,0,4,1,15024,0,55,37]},{"features":[43,2,72338,12,14,2,12,0,1,1,0,0,40,37]},{"features":[34,2,162604,11,9,4,2,2,2,1,0,0,40,37]},{"features":[20,2,258509,11,9,4,6,3,2,1,0,0,40,37]},{"features":[27,2,446947,9,13,4,0,4,2,0,0,0,55,37]},{"features":[20,2,95552,11,9,4,11,3,4,1,0,0,40,37]},{"features":[46,2,145636,11,9,2,3,0,4,1,3103,0,50,37]},{"features":[18,2,150675,0,6,4,11,3,4,1,0,0,40,37]},{"features":[22,2,197050,11,9,4,7,3,4,0,0,0,20,37]},{"features":[20,2,246635,15,10,4,11,3,4,0,2597,0,20,37]},{"features":[65,0,200764,11,9,6,0,1,4,0,0,0,40,37]},{"features":[38,2,175665,15,10,2,9,5,4,0,0,0,40,37]},{"features":[34,3,337995,9,13,0,3,4,2,1,15020,0,50,37]},{"features":[42,2,86912,9,13,0,7,1,4,1,0,0,40,37]},{"features":[40,2,100451,15,10,4,2,1,4,1,0,0,40,37]},{"features":[45,2,192360,12,14,2,3,0,4,1,0,1902,50,37]},{"features":[55,2,150507,15,10,2,0,0,4,1,0,0,40,37]},{"features":[36,2,48976,9,13,2,11,5,4,0,0,0,40,37]},{"features":[34,2,111567,15

Here are the headers of the train dataset. "Target" is the header of the ground truth label, and the others are the feature headers. They will be used to beautify the analysis report.

In [7]:
all_headers = [
    "Age",
    "Workclass",
    "fnlwgt",
    "Education",
    "Education-Num",
    "Marital Status",
    "Occupation",
    "Relationship",
    "Ethnic group",
    "Sex",
    "Capital Gain",
    "Capital Loss",
    "Hours per week",
    "Country",
    "Target",
]

To verify that the execution role for this notebook has the necessary permissions to proceed, put a simple test object into the S3 bucket specified above. If this command fails, update the role to have `s3:PutObject` permission on the bucket and try again.

In [8]:
sagemaker.s3.S3Uploader.upload_string_as_file_body(
    body="hello",
    desired_s3_uri=f"{s3_key}/upload-test-file.txt",
    sagemaker_session=sagemaker_session,
)
print("Success! We are all set to proceed with uploading to S3.")

Success! We are all set to proceed with uploading to S3.


Then upload the data files to S3 so that they can be used by SageMaker.

In [9]:
train_data_s3_uri = sagemaker.s3.S3Uploader.upload(
    local_path=train_dataset_path,
    desired_s3_uri=s3_key,
    sagemaker_session=sagemaker_session,
)
print(f"Train data is uploaded to: {train_data_s3_uri}")
test_data_s3_uri = sagemaker.s3.S3Uploader.upload(
    local_path=test_dataset_path,
    desired_s3_uri=s3_key,
    sagemaker_session=sagemaker_session,
)
print(f"Test data is uploaded to: {test_data_s3_uri}")

Train data is uploaded to: s3://sagemaker-us-west-2-678264136642/sagemaker/DEMO-ClarifyModelMonitor-1705692244-1224/validation-dataset.json
Test data is uploaded to: s3://sagemaker-us-west-2-678264136642/sagemaker/DEMO-ClarifyModelMonitor-1705692244-1224/test-dataset.json


### SageMaker model

This example includes a prebuilt [SageMaker Linear Learner](https://docs.aws.amazon.com/sagemaker/latest/dg/linear-learner.html) model trained by [a SageMaker Clarify offline processing example notebook](https://github.com/aws/amazon-sagemaker-examples/blob/main/sagemaker-clarify/fairness_and_explainability/fairness_and_explainability_jsonlines_format.ipynb). The model supports [SageMaker JSON Lines dense format](https://docs.aws.amazon.com/sagemaker/latest/dg/cdf-inference.html#common-in-formats) (MIME type `"application/jsonlines"`).

* The model input can one or more lines, each line is a JSON object that has a "features" key pointing to a list of feature values concerning demographic characteristics of individuals. For example,

```
{"features":[28,2,133937,9,13,2,0,0,4,1,15024,0,55,37]}
{"features":[43,2,72338,12,14,2,12,0,1,1,0,0,40,37]}
```

* The model output has the predictions of whether a person has a yearly income that is more than $50,000. Each prediction is a JSON object that has a "predicted_label" key pointing to the predicted label, and the "score" key pointing to the confidence score. For example,

```
{"predicted_label":1,"score":0.989977359771728}
{"predicted_label":1,"score":0.504138827323913}
```

In [10]:
model_file = "model/ll-adult-prediction-model.tar.gz"
model_url = sagemaker.s3.S3Uploader.upload(
    local_path=model_file,
    desired_s3_uri=s3_key,
    sagemaker_session=sagemaker_session,
)
print(f"Model file has been uploaded to {model_url}")

model_name = sagemaker.utils.unique_name_from_base("DEMO-xgb-churn-pred-model-monitor")
print(f"SageMaker model name: {model_name}")

image_uri = sagemaker.image_uris.retrieve("linear-learner", region, "1")
print(f"SageMaker Linear Learner image: {image_uri}")

model = sagemaker.model.Model(image_uri=image_uri, model_data=model_url, role=role)
container_def = model.prepare_container_def()
sagemaker_session.create_model(model_name, role, container_def)
print("SageMaker model created")

Model file has been uploaded to s3://sagemaker-us-west-2-678264136642/sagemaker/DEMO-ClarifyModelMonitor-1705692244-1224/ll-adult-prediction-model.tar.gz
SageMaker model name: DEMO-xgb-churn-pred-model-monitor-1705692245-0c05
SageMaker Linear Learner image: 174872318107.dkr.ecr.us-west-2.amazonaws.com/linear-learner:1
SageMaker model created


## Batch Transform Job

For continuous monitoring, batch transform jobs should be executed regularly with the latest data. But for demonstration purpose, the following cell only executes the job once before the monitor is scheduled, so that the first monitoring execution has captured data to process. 

See [Transformer](https://sagemaker.readthedocs.io/en/stable/api/inference/transformer.html#sagemaker.transformer.Transformer.transform) for the API reference. Highlights,

* `destination_s3_uri` is used to specify the data capture S3 URI which is a key connection between the job and the monitor.
* `join_source` must be set to "Input" for the transform output to include predictions (model output) as well as features (model input), because model bias monitor requires both.
* `generate_inference_id` must be set to True for the transform output to include a unique ID for each record. Model bias monitor requires both predicted labels and ground truth labels, so it needs the ID to join the captured data and the ground truth data.

**NOTE**: The following cell takes about 5 minutes to run.

In [11]:
transfomer = model.transformer(
    instance_count=1,
    instance_type="ml.m5.xlarge",
    accept=dataset_type,  # The transform output data format
    assemble_with="None",  # JSON records are under a single JSON structure, but this is required if join_source is set
    output_path=transform_output_s3_uri,
)

transfomer.transform(
    data=test_data_s3_uri,
    content_type=dataset_type,  # The transform input format
    split_type="None",  # JSON records are under a single JSON structure, but this is required if join_source is set
    join_source="Input",  # Include model input (features) in transform output
    batch_data_capture_config=sagemaker.inputs.BatchDataCaptureConfig(
        destination_s3_uri=data_capture_s3_uri,
        generate_inference_id=True,  # Inference ID is mandatory to join the captured data and the ground truth data
    ),
    wait=True,  # In real world you don't have to wait, but for demo purpose we wait for the output
    logs=False,  # You can change it to True to view job logs inline
)

INFO:sagemaker:Creating transform job with name: linear-learner-2024-01-19-19-24-07-189


................................................................!


### Captured data

Once the transform job completed, an "output" folders is created under `data_capture_s3_uri`, to includes the captured data files of transform output. Note that, batch transform data capture is unlike endpoint data capture, it does not capture the data for real as it will create tremendous amount of duplications. Instead, it generates [manifest](https://docs.aws.amazon.com/sagemaker/latest/APIReference/API_S3DataSource.html#sagemaker-Type-S3DataSource-S3Uri) files which refer to the transform output S3 location.

Now list the captured data files stored in Amazon S3. There should be different files from different time periods organized based on the hour in which the batch transformation occurred. The format of the Amazon S3 path is:

`s3://{data_capture_s3_uri}/output/yyyy/mm/dd/hh/filename.jsonl`

In [12]:
data_capture_output = f"{data_capture_s3_uri}/output"
captured_data_files = sorted(
    sagemaker.s3.S3Downloader.list(
        s3_uri=data_capture_output,
        sagemaker_session=sagemaker_session,
    )
)
print("Found capture data files:")
print("\n ".join(captured_data_files[-5:]))

Found capture data files:
s3://sagemaker-us-west-2-678264136642/sagemaker/DEMO-ClarifyModelMonitor-1705692244-1224/data-capture/output/2024/01/19/19/5ef12a7a-2c09-4d6f-817b-84823cee935f.json


In [13]:
data_capture_output_dict = json.loads(
    sagemaker.s3.S3Downloader.read_file(
        s3_uri=captured_data_files[-1],
        sagemaker_session=sagemaker_session,
    )
)
print(json.dumps(data_capture_output_dict, indent=4))

[
    {
        "prefix": "s3://sagemaker-us-west-2-678264136642/sagemaker/DEMO-ClarifyModelMonitor-1705692244-1224/transform-output/"
    },
    "test-dataset.json.out"
]


### Transform output

The captured data file refers to the transform output `.out` file.

In [14]:
transform_output = os.path.join(data_capture_output_dict[0]["prefix"], data_capture_output_dict[1])
transform_output

's3://sagemaker-us-west-2-678264136642/sagemaker/DEMO-ClarifyModelMonitor-1705692244-1224/transform-output/test-dataset.json.out'

View the content of the capture file.

In [15]:
transform_output_content = sagemaker.s3.S3Downloader.read_file(
    s3_uri=transform_output,
    sagemaker_session=sagemaker_session,
)
print(transform_output_content, sep="\n")

{"SageMakerInferenceId":"94bcc22a-0462-4bd1-92c9-b46a5aaec1aa","SageMakerInferenceTime":"2024-01-19T19:25:34Z","SageMakerOutput":{"predictions":[{"predicted_label":1,"score":0.9899773597717285},{"predicted_label":1,"score":0.5041388273239136},{"predicted_label":0,"score":0.06010060757398605},{"predicted_label":0,"score":0.03134893625974655},{"predicted_label":0,"score":0.09185617417097092},{"predicted_label":0,"score":0.03739730641245842},{"predicted_label":1,"score":0.49729207158088684},{"predicted_label":0,"score":0.008392381481826305},{"predicted_label":0,"score":0.00879521481692791},{"predicted_label":0,"score":0.029289718717336655},{"predicted_label":0,"score":0.08575712144374847},{"predicted_label":0,"score":0.06663481891155243},{"predicted_label":1,"score":0.9876857995986938},{"predicted_label":1,"score":0.5606499314308167},{"predicted_label":0,"score":0.1535872220993042},{"predicted_label":1,"score":0.8834722638130188},{"predicted_label":0,"score":0.383236825466156},{"predicted

The contents of a single line is present below in formatted JSON to observe a little better.

* The features are captured because the `join_source` parameter is set to "Input".
* The predictions are captured into the `"SageMakerOutput"` field.
* The inference ID and inference time (the start time of the transform job) are also captured because the `generate_inference_id` parameter is set to True.

In [16]:
# To pretty print the Transform output, uncomment below.  Warning: this could result in a very long log!
# print(json.dumps(json.loads(transform_output_content), indent=4))

## Ground Truth Data

Besides captured data, bias drift monitoring execution also requires ground truth data. In real use cases, you should regularly label the captured data, then upload the ground truth data (labels) to designated S3 location. For demonstration purpose, this example notebook generates fake ground truth data following [this schema](https://docs.aws.amazon.com/sagemaker/latest/dg/model-monitor-model-quality-merge.html), and then uploads it to `ground_truth_s3_uri` which is another key input to the monitor. The bias drift monitoring execution will first merge the captured data and the ground truth data, and then do bias analysis for the merged data.

Notice the value of the `data` field in `groundTruthData` **must be in the same format as how the ground truth labels are stored in the input dataset**.

In [17]:
def ground_truth_with_id(seeds, inference_id):
    instances = []
    for seed in seeds:
        random.seed(seed)  # to get consistent results
        label = (
            1 if random.random() < 0.7 else 0
        )  # randomly generate positive labels 70% of the time
        instances.append(
            {"label": label}
        )  # Also use the "label" key, the same as in the input dataset.
    # format required by the merge job and bias monitoring job
    return {
        "groundTruthData": {
            "data": json.dumps({"instances": instances}),
            "encoding": "JSON",
        },
        "eventMetadata": {
            "eventId": str(inference_id),
        },
        "eventVersion": "0",
    }


def upload_ground_truth(upload_time, upload_path, seeds, inference_id):
    # Single JSON object, containing all records
    fake_data = [ground_truth_with_id(seeds, inference_id)]
    data_to_upload = json.dumps(fake_data)
    target_s3_uri = f"{upload_path}/{upload_time:%Y/%m/%d/%H/%M%S}.jsonl"
    print(f"Uploading {len(seeds)} records to", target_s3_uri)
    sagemaker.s3.S3Uploader.upload_string_as_file_body(
        body=data_to_upload,
        desired_s3_uri=target_s3_uri,
        sagemaker_session=sagemaker_session,
    )

In [18]:
now = datetime.datetime.utcnow()
# Use unique IDs for each record.  JSON differs from JSONLines in that a single InferenceId can have multiple records,
# so we use arbitrary IDs so we can generate the ground truth labels consistently for each inference ID.
inference_id = json.loads(transform_output_content)["SageMakerInferenceId"]
seeds = [i for i, _record in enumerate(json.loads(transform_output_content)["instances"])]
print(inference_id)
print(seeds)
# Generate data for the last hour, in case the first monitoring execution is in this hour
upload_ground_truth(
    upload_time=now - datetime.timedelta(hours=1),
    upload_path=ground_truth_s3_uri,
    seeds=seeds,
    inference_id=inference_id,
)
# Generate data for this hour, in case the first monitoring execution will be in the next hour
upload_ground_truth(
    upload_time=now,
    upload_path=ground_truth_s3_uri,
    seeds=seeds,
    inference_id=inference_id,
)

94bcc22a-0462-4bd1-92c9-b46a5aaec1aa
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 203, 204, 205, 206, 207, 208, 209, 210, 211, 212, 213, 21

## Model Bias Monitor

Similar to the other monitoring types, the standard procedure of creating a [bias drift monitor](https://docs.aws.amazon.com/sagemaker/latest/dg/clarify-model-monitor-bias-drift.html) is first run a baselining job, and then schedule the monitor.

A bias drift monitoring execution starts a merge job that joins the captured data and ground truth data together using the inference ID. Then a SageMaker Clarify bias analysis job is started to compute all the [pre-training bias metrics](https://docs.aws.amazon.com/sagemaker/latest/dg/clarify-measure-data-bias.html) and [post-training bias metrics](https://docs.aws.amazon.com/sagemaker/latest/dg/clarify-measure-post-training-bias.html). on the merged data. The max execution time is divided equally between two jobs, the notebook is scheduling an hourly model bias monitor, so the `max_runtime_in_seconds` parameter should not exceed 1800 seconds.

In [19]:
model_bias_monitor = sagemaker.model_monitor.ModelBiasMonitor(
    role=role,
    sagemaker_session=sagemaker_session,
    max_runtime_in_seconds=1800,
)

INFO:sagemaker.image_uris:Defaulting to the only supported framework/algorithm version: 1.0.
INFO:sagemaker.image_uris:Ignoring unnecessary instance type: None.


### Baselining job

A baselining job runs predictions on training dataset and suggests constraints. The `suggest_baseline()` method of `ModelBiasMonitor` starts a SageMaker Clarify processing job to generate the constraints.

The step is not mandatory, but providing constraints file to the monitor can enable violations file generation.

#### Configurations

Information about the input data need to be provided to the processor.

`DataConfig` stores information about the dataset to be analyzed. For example, the dataset file and its format (like JSON Lines), where to store the analysis results. Some special things to note about this configuration for the JSON Lines dataset,

* The parameter value `"features"` or `"label"` is **NOT** a header string. Instead, it is a `JMESPath` expression ([refer to its specification](https://jmespath.org/specification.html)) that is used to locate the features list or the ground truth label in the dataset. In this example notebook they happen to be the same as the keys in the dataset. But for example, if the dataset has records like below, then the `features` parameter should use value `"data.features.values"`, and the `label` parameter should use value `"data.label"`.

  ```
  {"data": {"features": {"values": [25, 2, 226802, 1, 7, 4, 6, 3, 2, 1, 0, 0, 40, 37]}, "label": 0}}
  ```

* SageMaker Clarify processing job will load the JSON Lines dataset into tabular representation for further analysis, and the parameter `headers` is the list of column names. **The label header shall be the last one in the headers list**, and the order of feature headers shall be the same as the order of features in a record.

In [20]:
features_jmespath = "instances[*].features"
ground_truth_label_jmespath = "instances[*].label"
data_config = sagemaker.clarify.DataConfig(
    s3_data_input_path=train_data_s3_uri,
    s3_output_path=baselining_output_s3_uri,
    features=features_jmespath,
    label=ground_truth_label_jmespath,
    headers=all_headers,
    dataset_type=dataset_type,
)

`ModelConfig` is configuration related to model to be used for inferencing. In order to compute post-training bias metrics, the computation needs to get inferences for the SageMaker model. To accomplish this, the processing job will use the model to create an ephemeral endpoint (also known as "shadow endpoint"). The processing job will delete the shadow endpoint after the computations are completed. One special thing to note about this configuration for the JSON Lines model input and output,

* `content_template` is used by SageMaker Clarify processing job to convert the tabular data to the request payload acceptable to the shadow endpoint. To be more specific, the placeholder `$features` will be replaced by **the features list** from records. The request payload of a record from the testing dataset happens to be similar to the record itself, like `{"features":[28,2,133937,9,13,2,0,0,4,1,15024,0,55,37]}`, because both the dataset and the model input conform to the same format.

In [21]:
model_config = sagemaker.clarify.ModelConfig(
    model_name=model_name,  # The name of the SageMaker model
    instance_type="ml.m5.xlarge",  # The instance type of the shadow endpoint
    instance_count=1,  # The instance count of the shadow endpoint
    content_type=dataset_type,  # The data format of the model input
    accept_type=dataset_type,  # The data format of the model output
    content_template='{"instances":$records}',
    record_template='{"features":$features}',
)

`ModelPredictedLabelConfig` specifies how to extract predicted label from the model output. The example model returns the predicted label as well as the confidence score, so there are two ways to define this configuration,

* Set the `label` parameter to "predicted_label" which is the `JMESPath` expression to locate the predicted label in the model output. This is the way used in this example.
* Alternatively, you can set the `probability` parameter to "score" which is the `JMESPath` expression to locate the confidence score in the model output. And set the `probability_threshold` parameter to a floating number in between 0 and 1. The post-training analysis will use it to convert a score to binary predicted label (`0` or `1`). The default value is 0.5, which means a probability value > 0.5 indicates predicted label `1`.

In [22]:
predicted_label_jmespath = "predictions[*].predicted_label"
probability_jmespath = "predictions[*].score"
model_predicted_label_config = sagemaker.clarify.ModelPredictedLabelConfig(
    label=predicted_label_jmespath,
    probability=probability_jmespath,
)

`BiasConfig` is the configuration of the sensitive groups in the dataset. Typically, bias is measured by computing a metric and comparing it across groups.

  * The group of interest is specified using the facet parameters. With the following configuration, the baselining job will check for bias in the model's predictions with respect to gender and income. Specifically, it is checking if the model is more likely to predict that males have an annual income of over $50,000 compared to females. Although not demonstrated in this example, a bias monitor can measure bias against multiple sensitive attributes, if you provide a list of facets.
  * The `group_name` parameter is used to form subgroups for the measurement of [Conditional Demographic Disparity in Labels](https://docs.aws.amazon.com/sagemaker/latest/dg/clarify-data-bias-metric-cddl.html) (CDDL) and [Conditional Demographic Disparity in Predicted Labels](https://docs.aws.amazon.com/sagemaker/latest/dg/clarify-post-training-bias-metric-cddpl.html) (CDDPL) with regard to [Simpson’s paradox](https://en.wikipedia.org/wiki/Simpson%27s_paradox).

In [23]:
bias_config = sagemaker.clarify.BiasConfig(
    label_values_or_threshold=[1],  # the positive outcome is earning >$50,000
    facet_name="Sex",  # the sensitive attribute is the gender
    facet_values_or_threshold=[0],  # the disadvantaged group is female
    group_name="Age",
)

#### Kick off baselining job

Call the `suggest_baseline()` method to start the baselining job. The job computes all the [pre-training bias metrics](https://docs.aws.amazon.com/sagemaker/latest/dg/clarify-measure-data-bias.html) and [post-training bias metrics](https://docs.aws.amazon.com/sagemaker/latest/dg/clarify-measure-post-training-bias.html).

In [24]:
model_bias_monitor.suggest_baseline(
    bias_config=bias_config,
    data_config=data_config,
    model_config=model_config,
    model_predicted_label_config=model_predicted_label_config,
)

INFO:sagemaker.image_uris:Defaulting to the only supported framework/algorithm version: 1.0.
INFO:sagemaker.image_uris:Ignoring unnecessary instance type: None.
INFO:sagemaker.clarify:Analysis Config: {'dataset_type': 'application/json', 'features': 'instances[*].features', 'headers': ['Age', 'Workclass', 'fnlwgt', 'Education', 'Education-Num', 'Marital Status', 'Occupation', 'Relationship', 'Ethnic group', 'Sex', 'Capital Gain', 'Capital Loss', 'Hours per week', 'Country', 'Target'], 'label': 'instances[*].label', 'label_values_or_threshold': [1], 'facet': [{'name_or_index': 'Sex', 'value_or_threshold': [0]}], 'group_variable': 'Age', 'methods': {'report': {'name': 'report', 'title': 'Analysis Report'}, 'pre_training_bias': {'methods': 'all'}, 'post_training_bias': {'methods': 'all'}}, 'predictor': {'model_name': 'DEMO-xgb-churn-pred-model-monitor-1705692245-0c05', 'instance_type': 'ml.m5.xlarge', 'initial_instance_count': 1, 'accept_type': 'application/json', 'content_type': 'applica

**NOTE**: The following cell waits until the baselining job is completed (in about 10 minutes). It then inspects the suggested constraints. This step can be skipped, because the monitor to be scheduled will automatically pick up baselining job name and wait for it before monitoring execution.

In [25]:
model_bias_monitor.latest_baselining_job.wait(logs=False)
print()
model_bias_constraints = model_bias_monitor.suggested_constraints()
print(f"Suggested constraints: {model_bias_constraints.file_s3_uri}")
print(
    sagemaker.s3.S3Downloader.read_file(
        s3_uri=model_bias_constraints.file_s3_uri,
        sagemaker_session=sagemaker_session,
    )
)

.......................................................................................................................!
Suggested constraints: s3://sagemaker-us-west-2-678264136642/sagemaker/DEMO-ClarifyModelMonitor-1705692244-1224/baselining-output/analysis.json
{
    "version": "1.0",
    "post_training_bias_metrics": {
        "label": "Target",
        "facets": {
            "Sex": [
                {
                    "value_or_threshold": "0",
                    "metrics": [
                        {
                            "name": "AD",
                            "description": "Accuracy Difference (AD)",
                            "value": -0.15156641604010024
                        },
                        {
                            "name": "CDDPL",
                            "description": "Conditional Demographic Disparity in Predicted Labels (CDDPL)",
                            "value": 0.28176563733194276
                        },
                      

### Monitoring Schedule

With above constraints collected, now call `create_monitoring_schedule()` method to schedule an hourly model bias monitor.

If a baselining job has been submitted, then the monitor object will automatically pick up the analysis configuration from the baselining job. But if the baselining step is skipped, or if the capture dataset has different nature than the training dataset, then analysis configuration has to be provided.

`BiasAnalysisConfig` is a subset of the configuration of the baselining job, many options are not needed because,

* Model bias monitor will merge the captured data and the ground truth data, and then use the merged data as the input dataset.
* Capture data already includes predictions, so there is no need to create shadow endpoint.
* Attributes like probability threshold are provided as part of `BatchTransformInput`.

Highlights,

* `data_capture_s3_uri` is the location of data captured by the batch transform job
* `ground_truth_s3_uri` is the location of ground truth data
* `features_attribute` is the `JMESPath` expression to locate the features in model input, similar to the `features` parameter of `DataConfig`.
* `inference_attribute` is the `JMESPath` expression to locate the predicted label in model output, similar to the `label` parameter of `ModelPredictedLabelConfig`.

In [26]:
schedule_expression = sagemaker.model_monitor.CronExpressionGenerator.hourly()

In [27]:
model_bias_analysis_config = None

model_bias_analysis_config = sagemaker.model_monitor.BiasAnalysisConfig(
    bias_config,
    headers=all_headers,
    label=ground_truth_label_jmespath,
)
model_bias_monitor.create_monitoring_schedule(
    analysis_config=model_bias_analysis_config,
    batch_transform_input=sagemaker.model_monitor.BatchTransformInput(
        data_captured_destination_s3_uri=data_capture_s3_uri,
        destination="/opt/ml/processing/transform",
        dataset_format=sagemaker.model_monitor.MonitoringDatasetFormat.json(lines=False),
        features_attribute=features_jmespath,  # mandatory if no baselining job
        inference_attribute=predicted_label_jmespath,  # mandatory if no baselining job
        # look back 6 hour for transform job output.
        start_time_offset="-PT6H",
        end_time_offset="-PT0H",
    ),
    ground_truth_input=ground_truth_s3_uri,
    output_s3_uri=monitor_output_s3_uri,
    schedule_cron_expression=schedule_expression,
)
print(f"Model bias monitoring schedule: {model_bias_monitor.monitoring_schedule_name}")

INFO:sagemaker.model_monitor.clarify_model_monitoring:Uploading analysis config to {s3_uri}.
INFO:sagemaker.model_monitor.model_monitoring:Creating Monitoring Schedule with name: monitoring-schedule-2024-01-19-19-39-39-971


Model bias monitoring schedule: monitoring-schedule-2024-01-19-19-39-39-971


#### Wait for the first execution

The schedule starts jobs at the previously specified intervals. Code below waits until time crosses the hour boundary (in UTC) to see executions kick off.

Note: Even for an hourly schedule, Amazon SageMaker has a buffer period of 20 minutes to schedule executions. The execution might start in anywhere from zero to ~20 minutes from the hour boundary. This is expected and done for load balancing in the backend.

In [28]:
def wait_for_execution_to_start(model_monitor):
    print(
        "An hourly schedule was created above and it will kick off executions ON the hour (plus 0 - 20 min buffer)."
    )

    print("Waiting for the first execution to happen", end="")
    schedule_desc = model_monitor.describe_schedule()
    while "LastMonitoringExecutionSummary" not in schedule_desc:
        schedule_desc = model_monitor.describe_schedule()
        print(".", end="", flush=True)
        time.sleep(60)
    print()
    print("Done! Execution has been created")

    print("Now waiting for execution to start", end="")
    while schedule_desc["LastMonitoringExecutionSummary"]["MonitoringExecutionStatus"] in "Pending":
        schedule_desc = model_monitor.describe_schedule()
        print(".", end="", flush=True)
        time.sleep(10)

    print()
    print("Done! Execution has started")

**NOTE**: The following cell waits until the first monitoring execution is started. As explained above, the wait could take more than 60 minutes.

In [29]:
wait_for_execution_to_start(model_bias_monitor)

An hourly schedule was created above and it will kick off executions ON the hour (plus 0 - 20 min buffer).
Waiting for the first execution to happen............................
Done! Execution has been created
Now waiting for execution to start....
Done! Execution has started


In real world, a monitoring schedule is supposed to be active all the time. But in this example, it can be stopped to avoid incurring extra charges. A stopped schedule will not trigger further executions, but the ongoing execution will continue. And if needed, the schedule can be restarted by `start_monitoring_schedule()`.

In [30]:
model_bias_monitor.stop_monitoring_schedule()

INFO:sagemaker:Stopping Monitoring Schedule with name: monitoring-schedule-2024-01-19-19-39-39-971


#### Wait for the execution to finish

In the previous cell, the first execution has started. This section waits for the execution to finish so that its analysis results are available. Here are the possible terminal states and what each of them mean:

* `Completed` - This means the monitoring execution completed, and no issues were found in the violations report.
* `CompletedWithViolations` - This means the execution completed, but constraint violations were detected.
* `Failed` - The monitoring execution failed, maybe due to client error (perhaps incorrect role permissions) or infrastructure issues. Further examination of `FailureReason` and `ExitMessage` is necessary to identify what exactly happened.
* `Stopped` - job exceeded max runtime or was manually stopped.

In [31]:
# Waits for the schedule to have last execution in a terminal status.
def wait_for_execution_to_finish(model_monitor):
    schedule_desc = model_monitor.describe_schedule()
    execution_summary = schedule_desc.get("LastMonitoringExecutionSummary")
    if execution_summary is not None:
        print("Waiting for execution to finish", end="")
        while execution_summary["MonitoringExecutionStatus"] not in [
            "Completed",
            "CompletedWithViolations",
            "Failed",
            "Stopped",
        ]:
            print(".", end="", flush=True)
            time.sleep(60)
            schedule_desc = model_monitor.describe_schedule()
            execution_summary = schedule_desc["LastMonitoringExecutionSummary"]
        print()
        print(f"Done! Execution Status: {execution_summary['MonitoringExecutionStatus']}")
    else:
        print("Last execution not found")

**NOTE**: The following cell takes about 10 minutes.

In [32]:
wait_for_execution_to_finish(model_bias_monitor)

Waiting for execution to finish...........
Done! Execution Status: CompletedWithViolations


#### Merged data

Merged data is the intermediate results of bias drift monitoring execution. It is saved to JSON Lines files under the "merge" folder of `monitor_output_s3_uri`. Each line is a valid JSON object which combines the captured data and the ground truth data.

In [33]:
merged_data_s3_uri = f"{monitor_output_s3_uri}/merge"
print(merged_data_s3_uri)
merged_data_files = sorted(
    sagemaker.s3.S3Downloader.list(
        s3_uri=merged_data_s3_uri,
        sagemaker_session=sagemaker_session,
    )
)
print("Found merged data files:")
print("\n ".join(merged_data_files[-5:]))

s3://sagemaker-us-west-2-678264136642/sagemaker/DEMO-ClarifyModelMonitor-1705692244-1224/monitor-output/merge
Found merged data files:
s3://sagemaker-us-west-2-678264136642/sagemaker/DEMO-ClarifyModelMonitor-1705692244-1224/monitor-output/merge/monitoring-schedule-2024-01-19-19-39-39-971/2024/01/19/19/part-00000-8cf95ad5-2469-48d9-a3dc-dd5a17fc8823.c000.jsonl


The following cell prints a single line of a merged data file.

* `eventId` is the inference ID from the captured data and the ground truth data
* `groundTruthData` is from the ground truth data
* `captureData` is from the captured data. In this case, the `data` of `batchTransformOutput` is from the transform output.

In [34]:
merged_record = sagemaker.s3.S3Downloader.read_file(
    s3_uri=merged_data_files[-1],
    sagemaker_session=sagemaker_session,
).splitlines()[0]
print(json.dumps(json.loads(merged_record), indent=4))

{
    "eventMetadata": {
        "eventId": "94bcc22a-0462-4bd1-92c9-b46a5aaec1aa"
    },
    "eventVersion": "0",
    "groundTruthData": {
        "data": "{\"instances\": [{\"label\": 0}, {\"label\": 1}, {\"label\": 0}, {\"label\": 1}, {\"label\": 1}, {\"label\": 1}, {\"label\": 0}, {\"label\": 1}, {\"label\": 1}, {\"label\": 1}, {\"label\": 1}, {\"label\": 1}, {\"label\": 1}, {\"label\": 1}, {\"label\": 1}, {\"label\": 0}, {\"label\": 1}, {\"label\": 1}, {\"label\": 1}, {\"label\": 1}, {\"label\": 0}, {\"label\": 1}, {\"label\": 0}, {\"label\": 0}, {\"label\": 0}, {\"label\": 1}, {\"label\": 0}, {\"label\": 1}, {\"label\": 1}, {\"label\": 1}, {\"label\": 1}, {\"label\": 1}, {\"label\": 1}, {\"label\": 1}, {\"label\": 1}, {\"label\": 1}, {\"label\": 1}, {\"label\": 1}, {\"label\": 1}, {\"label\": 1}, {\"label\": 1}, {\"label\": 1}, {\"label\": 1}, {\"label\": 1}, {\"label\": 1}, {\"label\": 1}, {\"label\": 0}, {\"label\": 1}, {\"label\": 1}, {\"label\": 1}, {\"label\": 1}, {\"label\"

#### Inspect execution results

List the generated reports,

* analysis.json includes all the bias metrics.
* report.* files are static report files to visualize the bias metrics

In [35]:
schedule_desc = model_bias_monitor.describe_schedule()
execution_summary = schedule_desc.get("LastMonitoringExecutionSummary")
if execution_summary and execution_summary["MonitoringExecutionStatus"] in [
    "Completed",
    "CompletedWithViolations",
]:
    last_model_bias_monitor_execution = model_bias_monitor.list_executions()[-1]
    last_model_bias_monitor_execution_report_uri = (
        last_model_bias_monitor_execution.output.destination
    )
    print(f"Report URI: {last_model_bias_monitor_execution_report_uri}")
    last_model_bias_monitor_execution_report_files = sorted(
        sagemaker.s3.S3Downloader.list(
            s3_uri=last_model_bias_monitor_execution_report_uri,
            sagemaker_session=sagemaker_session,
        )
    )
    print("Found Report Files:")
    print("\n ".join(last_model_bias_monitor_execution_report_files))
else:
    last_model_bias_monitor_execution = None
    print(
        "====STOP==== \n No completed executions to inspect further. Please wait till an execution completes or investigate previously reported failures."
    )
    print(schedule_desc)

Report URI: s3://sagemaker-us-west-2-678264136642/sagemaker/DEMO-ClarifyModelMonitor-1705692244-1224/monitor-output/monitoring-schedule-2024-01-19-19-39-39-971/2024/01/19/20
Found Report Files:
s3://sagemaker-us-west-2-678264136642/sagemaker/DEMO-ClarifyModelMonitor-1705692244-1224/monitor-output/monitoring-schedule-2024-01-19-19-39-39-971/2024/01/19/20/analysis.json
 s3://sagemaker-us-west-2-678264136642/sagemaker/DEMO-ClarifyModelMonitor-1705692244-1224/monitor-output/monitoring-schedule-2024-01-19-19-39-39-971/2024/01/19/20/constraint_violations.json
 s3://sagemaker-us-west-2-678264136642/sagemaker/DEMO-ClarifyModelMonitor-1705692244-1224/monitor-output/monitoring-schedule-2024-01-19-19-39-39-971/2024/01/19/20/report.html
 s3://sagemaker-us-west-2-678264136642/sagemaker/DEMO-ClarifyModelMonitor-1705692244-1224/monitor-output/monitoring-schedule-2024-01-19-19-39-39-971/2024/01/19/20/report.ipynb
 s3://sagemaker-us-west-2-678264136642/sagemaker/DEMO-ClarifyModelMonitor-1705692244-1224

If there are any violations compared to the baseline, they are listed here. See [Bias Drift Violations](https://docs.aws.amazon.com/sagemaker/latest/dg/clarify-model-monitor-bias-drift-violations.html) for the schema of the file, and how violations are detected.

In [36]:
violations = model_bias_monitor.latest_monitoring_constraint_violations()
if violations is not None:
    pprint.PrettyPrinter(indent=4).pprint(violations.body_dict)

{   'version': '1.0',
    'violations': [   {   'constraint_check_type': 'bias_drift_check',
                          'description': 'Metric value 0.374894782529513 '
                                         "doesn't meet the baseline constraint "
                                         'requirement 0.28176563733194276',
                          'facet': 'Sex',
                          'facet_value': '0',
                          'metric_name': 'CDDPL'},
                      {   'constraint_check_type': 'bias_drift_check',
                          'description': 'Metric value -0.34693877551020413 '
                                         "doesn't meet the baseline constraint "
                                         'requirement -0.09508196721311479',
                          'facet': 'Sex',
                          'facet_value': '0',
                          'metric_name': 'DAR'},
                      {   'constraint_check_type': 'bias_drift_check',
                     

By default, the analysis results are also published to CloudWatch, see [CloudWatch Metrics for Bias Drift Analysis](https://docs.aws.amazon.com/sagemaker/latest/dg/clarify-model-monitor-bias-drift-cw.html).

## Cleanup

If there is no plan to collect more data for bias drift monitoring, then the monitor should be stopped (and deleted) to avoid incurring additional charges. Note that deleting the monitor does not delete the data in S3.

In [37]:
model_bias_monitor.stop_monitoring_schedule()
wait_for_execution_to_finish(model_bias_monitor)
model_bias_monitor.delete_monitoring_schedule()

INFO:sagemaker:Stopping Monitoring Schedule with name: monitoring-schedule-2024-01-19-19-39-39-971
INFO:sagemaker:Deleting Monitoring Schedule with name: monitoring-schedule-2024-01-19-19-39-39-971


Waiting for execution to finish
Done! Execution Status: CompletedWithViolations


INFO:sagemaker.model_monitor.clarify_model_monitoring:Deleting Model Bias Job Definition with name: model-bias-job-definition-2024-01-19-19-39-39-971


In [38]:
sagemaker_session.delete_model(model_name)

INFO:sagemaker:Deleting model with name: DEMO-xgb-churn-pred-model-monitor-1705692245-0c05


## Notebook CI Test Results

This notebook was tested in multiple regions. The test results are as follows, except for us-west-2 which is shown at the top of the notebook.


![This us-east-1 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://prod.us-west-2.tcx-beacon.docs.aws.dev/sagemaker-nb/us-east-1/deploy_and_monitor|sm-clarify_model_bias_monitor_batch_transform_json|sm-clarify_model_bias_monitor_batch_transform_json.ipynb)

![This us-east-2 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://prod.us-west-2.tcx-beacon.docs.aws.dev/sagemaker-nb/us-east-2/deploy_and_monitor|sm-clarify_model_bias_monitor_batch_transform_json|sm-clarify_model_bias_monitor_batch_transform_json.ipynb)

![This us-west-1 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://prod.us-west-2.tcx-beacon.docs.aws.dev/sagemaker-nb/us-west-1/deploy_and_monitor|sm-clarify_model_bias_monitor_batch_transform_json|sm-clarify_model_bias_monitor_batch_transform_json.ipynb)

![This ca-central-1 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://prod.us-west-2.tcx-beacon.docs.aws.dev/sagemaker-nb/ca-central-1/deploy_and_monitor|sm-clarify_model_bias_monitor_batch_transform_json|sm-clarify_model_bias_monitor_batch_transform_json.ipynb)

![This sa-east-1 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://prod.us-west-2.tcx-beacon.docs.aws.dev/sagemaker-nb/sa-east-1/deploy_and_monitor|sm-clarify_model_bias_monitor_batch_transform_json|sm-clarify_model_bias_monitor_batch_transform_json.ipynb)

![This eu-west-1 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://prod.us-west-2.tcx-beacon.docs.aws.dev/sagemaker-nb/eu-west-1/deploy_and_monitor|sm-clarify_model_bias_monitor_batch_transform_json|sm-clarify_model_bias_monitor_batch_transform_json.ipynb)

![This eu-west-2 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://prod.us-west-2.tcx-beacon.docs.aws.dev/sagemaker-nb/eu-west-2/deploy_and_monitor|sm-clarify_model_bias_monitor_batch_transform_json|sm-clarify_model_bias_monitor_batch_transform_json.ipynb)

![This eu-west-3 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://prod.us-west-2.tcx-beacon.docs.aws.dev/sagemaker-nb/eu-west-3/deploy_and_monitor|sm-clarify_model_bias_monitor_batch_transform_json|sm-clarify_model_bias_monitor_batch_transform_json.ipynb)

![This eu-central-1 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://prod.us-west-2.tcx-beacon.docs.aws.dev/sagemaker-nb/eu-central-1/deploy_and_monitor|sm-clarify_model_bias_monitor_batch_transform_json|sm-clarify_model_bias_monitor_batch_transform_json.ipynb)

![This eu-north-1 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://prod.us-west-2.tcx-beacon.docs.aws.dev/sagemaker-nb/eu-north-1/deploy_and_monitor|sm-clarify_model_bias_monitor_batch_transform_json|sm-clarify_model_bias_monitor_batch_transform_json.ipynb)

![This ap-southeast-1 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://prod.us-west-2.tcx-beacon.docs.aws.dev/sagemaker-nb/ap-southeast-1/deploy_and_monitor|sm-clarify_model_bias_monitor_batch_transform_json|sm-clarify_model_bias_monitor_batch_transform_json.ipynb)

![This ap-southeast-2 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://prod.us-west-2.tcx-beacon.docs.aws.dev/sagemaker-nb/ap-southeast-2/deploy_and_monitor|sm-clarify_model_bias_monitor_batch_transform_json|sm-clarify_model_bias_monitor_batch_transform_json.ipynb)

![This ap-northeast-1 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://prod.us-west-2.tcx-beacon.docs.aws.dev/sagemaker-nb/ap-northeast-1/deploy_and_monitor|sm-clarify_model_bias_monitor_batch_transform_json|sm-clarify_model_bias_monitor_batch_transform_json.ipynb)

![This ap-northeast-2 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://prod.us-west-2.tcx-beacon.docs.aws.dev/sagemaker-nb/ap-northeast-2/deploy_and_monitor|sm-clarify_model_bias_monitor_batch_transform_json|sm-clarify_model_bias_monitor_batch_transform_json.ipynb)

![This ap-south-1 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://prod.us-west-2.tcx-beacon.docs.aws.dev/sagemaker-nb/ap-south-1/deploy_and_monitor|sm-clarify_model_bias_monitor_batch_transform_json|sm-clarify_model_bias_monitor_batch_transform_json.ipynb)
